In [1]:
import keras
import numpy as np
import tensorflow as tf

from absl import flags
from pathlib import Path
from operator import itemgetter
from keras.optimizers import Adam
from sklearn.metrics import classification_report, recall_score, precision_score, f1_score

Using TensorFlow backend.


In [2]:
import importlib

import model as nn
importlib.reload(nn)

elsa_architecture = nn.elsa_architecture

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

lang = "ja"
batch_size = 250
lr = 1e-3
epochs = 100
patience = 3
data_dir = "/data/elsa2"
checkpoint_dir = "./ckpt"
optimizer = "adam"

In [ ]:
lstm_hidden = 512
lstm_drop = 0.5
final_drop = 0.5
embed_drop = 0.0
highway = False
compute_class_weight = False
multilabel = True

In [ ]:
data_dir = Path(data_dir)
wv_path = (data_dir / "{:s}_wv.npy".format(lang)).__str__()
X_path = (data_dir / "{:s}_X.npy".format(lang)).__str__()
y_path = (data_dir / "{:s}_y.npy".format(lang)).__str__()
emoji_path = (data_dir / "{:s}_emoji.txt".format(lang)).__str__()

wv = np.load(wv_path, allow_pickle=True)
input_vec = np.load(X_path, allow_pickle=True)
input_label = np.load(y_path, allow_pickle=True)

nb_tokens = len(wv)
embed_dim = wv.shape[1]
input_len = len(input_label)
nb_classes = input_label.shape[1]
maxlen = input_vec.shape[1]

train_end = int(input_len*0.7)
val_end = int(input_len*0.9)

(X_train, y_train) = (input_vec[:train_end], input_label[:train_end])
(X_val, y_val) = (input_vec[train_end:val_end], input_label[train_end:val_end])
(X_test, y_test) = (input_vec[val_end:], input_label[val_end:])

if multilabel:
    def to_multilabel(y):
        outputs = []
        for i in range(nb_classes):
            outputs.append(y[:, i])
        return outputs

    y_train = to_multilabel(y_train)
    y_val = to_multilabel(y_val)
    y_test = to_multilabel(y_test)

model = elsa_architecture(nb_classes=nb_classes,
                          nb_tokens=nb_tokens,
                          maxlen=maxlen,
                          final_dropout_rate=final_drop,
                          embed_dropout_rate=embed_drop,
                          load_embedding=True,
                          pre_embedding=wv,
                          high=highway,
                          embed_dim=embed_dim,
                          multilabel=multilabel)
model.summary()

computed_class_weight = None

if multilabel:
    loss = "binary_crossentropy"
else:
    loss = "categorical_crossentropy"
    if compute_class_weight:
        y_train_sps = []
        for row in y_train:
            y_train_sps.extend(np.where(row)[0].tolist())
        computed_class_weight = class_weight.compute_class_weight(
            'balanced', list(range(nb_classes)), y_train_sps)
        print("computed class weight = {:s}".format(str(computed_class_weight)))

if optimizer == 'adam':
    adam = Adam(clipnorm=1, lr=lr)
    model.compile(loss=loss, optimizer=adam, metrics=['accuracy'])
elif optimizer == 'rmsprop':
    model.compile(loss=loss, optimizer='rmsprop', metrics=['accuracy'])

checkpoint_dir = Path(checkpoint_dir)
if not checkpoint_dir.exists():
    checkpoint_dir.mkdir()
checkpoint_weight_path = (checkpoint_dir / "elsa_{:s}.hdf5".format(lang)).__str__()

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss', min_delta=0, patience=patience, verbose=0, mode='auto'),
    keras.callbacks.ModelCheckpoint(checkpoint_weight_path, monitor='val_loss',
                                    verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
]
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_val, y_val),
          class_weight=computed_class_weight,
          callbacks=callbacks,
          verbose=2)

freq = {line.split()[0]: int(line.split()[1]) for line in open(emoji_path).readlines()}
freq_topn = sorted(freq.items(), key=itemgetter(1), reverse=True)[:nb_classes]

if multilabel:
    y_pred = model.predict([X_test], batch_size=batch_size)
    y_pred = [np.squeeze(p) for p in y_pred]

    y_test_1d = np.array(y_test).flatten()
    y_pred_1d = np.array(y_pred).flatten()
    print(f1_score(y_test_1d, y_pred_1d > 0.5))
    print(classification_report(y_test_1d, y_pred_1d > 0.5))

    gold, pred = [], []
    for i in range(len(X_test)):
        each_gold, each_pred = [], []
        for c in range(nb_classes):
            if y_test[c][i] == 1.0:
                each_gold.append(c+1)
            else:
                each_gold.append(0)
            if y_pred[c][i] > 0.5:
                each_pred.append(c+1)
            else:
                each_pred.append(0)
        gold.extend(each_gold)
        pred.extend(each_pred)

    target_name = [""] + [e[0] for e in freq_topn]
    print(classification_report(gold, pred, target_names=target_name))
else:
    _, acc = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
    print(acc)

    y_pred = model.predict(X_test)
    print(classification_report(y_test.argmax(axis=1), y_pred.argmax(
        axis=1), target_names=[e[0] for e in freq_topn]))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 50, 200)      65831000    input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 50, 200)      0           embedding[0][0]                  
__________________________________________________________________________________________________
bi_lstm_0 (Bidirectio

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 2277475 samples, validate on 650707 samples
Epoch 1/100
 - 5551s - loss: 5.3403 - sigmoid_0_loss: 0.3614 - sigmoid_1_loss: 0.2282 - sigmoid_2_loss: 0.2474 - sigmoid_3_loss: 0.1693 - sigmoid_4_loss: 0.2012 - sigmoid_5_loss: 0.2072 - sigmoid_6_loss: 0.1769 - sigmoid_7_loss: 0.1707 - sigmoid_8_loss: 0.1136 - sigmoid_9_loss: 0.1131 - sigmoid_10_loss: 0.1376 - sigmoid_11_loss: 0.1298 - sigmoid_12_loss: 0.1598 - sigmoid_13_loss: 0.0772 - sigmoid_14_loss: 0.0535 - sigmoid_15_loss: 0.0932 - sigmoid_16_loss: 0.1164 - sigmoid_17_loss: 0.1042 - sigmoid_18_loss: 0.1082 - sigmoid_19_loss: 0.0840 - sigmoid_20_loss: 0.1004 - sigmoid_21_loss: 0.0884 - sigmoid_22_loss: 0.0850 - sigmoid_23_loss: 0.0747 - sigmoid_24_loss: 0.0812 - sigmoid_25_loss: 0.0755 - sigmoid_26_loss: 0.0840 - sigmoid_27_loss: 0.0817 - sigmoid_28_loss: 0.0764 - sigmoid_29_loss: 0.0522 - sigmoid_30_lo

Epoch 3/100
 - 5059s - loss: 4.6593 - sigmoid_0_loss: 0.3196 - sigmoid_1_loss: 0.2047 - sigmoid_2_loss: 0.2206 - sigmoid_3_loss: 0.1435 - sigmoid_4_loss: 0.1742 - sigmoid_5_loss: 0.1853 - sigmoid_6_loss: 0.1613 - sigmoid_7_loss: 0.1466 - sigmoid_8_loss: 0.0968 - sigmoid_9_loss: 0.0922 - sigmoid_10_loss: 0.1208 - sigmoid_11_loss: 0.1187 - sigmoid_12_loss: 0.1401 - sigmoid_13_loss: 0.0644 - sigmoid_14_loss: 0.0406 - sigmoid_15_loss: 0.0820 - sigmoid_16_loss: 0.0973 - sigmoid_17_loss: 0.0908 - sigmoid_18_loss: 0.0902 - sigmoid_19_loss: 0.0757 - sigmoid_20_loss: 0.0920 - sigmoid_21_loss: 0.0733 - sigmoid_22_loss: 0.0701 - sigmoid_23_loss: 0.0591 - sigmoid_24_loss: 0.0724 - sigmoid_25_loss: 0.0636 - sigmoid_26_loss: 0.0744 - sigmoid_27_loss: 0.0689 - sigmoid_28_loss: 0.0676 - sigmoid_29_loss: 0.0380 - sigmoid_30_loss: 0.0646 - sigmoid_31_loss: 0.0538 - sigmoid_32_loss: 0.0250 - sigmoid_33_loss: 0.0153 - sigmoid_34_loss: 0.0068 - sigmoid_35_loss: 0.0402 - sigmoid_36_loss: 0.0116 - sigmoid_37

Epoch 5/100
 - 4826s - loss: 4.3701 - sigmoid_0_loss: 0.3017 - sigmoid_1_loss: 0.1942 - sigmoid_2_loss: 0.2093 - sigmoid_3_loss: 0.1339 - sigmoid_4_loss: 0.1638 - sigmoid_5_loss: 0.1752 - sigmoid_6_loss: 0.1518 - sigmoid_7_loss: 0.1369 - sigmoid_8_loss: 0.0905 - sigmoid_9_loss: 0.0851 - sigmoid_10_loss: 0.1117 - sigmoid_11_loss: 0.1123 - sigmoid_12_loss: 0.1304 - sigmoid_13_loss: 0.0594 - sigmoid_14_loss: 0.0365 - sigmoid_15_loss: 0.0763 - sigmoid_16_loss: 0.0908 - sigmoid_17_loss: 0.0839 - sigmoid_18_loss: 0.0835 - sigmoid_19_loss: 0.0713 - sigmoid_20_loss: 0.0848 - sigmoid_21_loss: 0.0677 - sigmoid_22_loss: 0.0640 - sigmoid_23_loss: 0.0539 - sigmoid_24_loss: 0.0668 - sigmoid_25_loss: 0.0582 - sigmoid_26_loss: 0.0678 - sigmoid_27_loss: 0.0634 - sigmoid_28_loss: 0.0615 - sigmoid_29_loss: 0.0342 - sigmoid_30_loss: 0.0597 - sigmoid_31_loss: 0.0499 - sigmoid_32_loss: 0.0221 - sigmoid_33_loss: 0.0135 - sigmoid_34_loss: 0.0058 - sigmoid_35_loss: 0.0367 - sigmoid_36_loss: 0.0102 - sigmoid_37